In [ ]:
import os
import numpy as np
from sklearn import svm
import joblib

def symbol_removal(email):
    e_mod=email
    sym=["\\","&","!",".",",","/","+","*","%","(",")",":",";","\"","_","?","=","@","{","}","[","]","#"]
    for s in sym:
        e_mod=e_mod.replace(s," ")
    e_mod.replace("\'","")
    e_mod=e_mod.replace("\n"," ")
    e_mod=e_mod.replace("-","")

    return e_mod

def lemmatisation(email,dic):
    
    for key in dic.keys():
        e_mod=email.replace(key," "+dic[key]+" ")
    
    return e_mod

def num_token(email):
    num=["0","1","2","3","4","5","6","7","8","9"]
    e_mod=email
    for n in num:
        e_mod=e_mod.replace(n," "+"0"+" ")
    
    return e_mod


def idf(bag,lis):
    idf_list=[]
    N=len(lis)
    for term in bag:
        a=0
        for doc in lis:
            if term in doc:
                a+=1
        if a==0:
            a=N
        idf_list.append(a)
    idf_vector=np.log(N/np.array(idf_list,dtype="float64"))
    return idf_vector

def tf(bag,doc):
    N=len(bag)
    tf_list=[]
    for i in range(N):
        count=doc.count(bag[i])
        tf_list.append(count/len(doc))
    tf_vector=np.array(tf_list,dtype="float64")
    return tf_vector

def tf_idf_vectorisation(bag,emails):
    idf_v=idf(bag,emails)
    N=len(bag)
    num_of_docs=len(emails)
    X=np.empty((N,num_of_docs))
    for i in range(num_of_docs):
        tf_v=tf(bag,emails[i])
        X[:,i]=tf_v*idf_v
    
    return X

def processing():           #function for processing the test emails
    email=[]
    directory="test" 
    for file in sorted(os.listdir(directory),key= lambda x: int(x.replace("email","").replace(".txt",""))):#reading in asc order
        f=open(directory+"/"+file,"r",encoding='cp437')
        b=f.readline().lower()
        b=b.replace("subject: fw :","")
        b=b.replace("subject: fw : fw :","")
        b=b.replace("subject: re : re :","")        #removing the the word "subject" from emails at beginning of emails
        b=b.replace("subject: re :","")
        b=b.replace("subject: ","")
        a=b+f.read().lower()
        email.append(a)
        
    lemma_dictionary=np.load("lemma_dictionary.npy",allow_pickle=True).tolist()  #loading the lemma dictionary

    for i in range(len(email)):
        email[i]=symbol_removal(email[i])
        email[i]=num_token(email[i])
        email[i]=lemmatisation(email[i],lemma_dictionary)   #removing punctations, numbers and lemmatising the words

    length=len(email)
    email_bag=np.load("bagofwords.npy").tolist()        #loading the bag_of_words 

    emails_list=[]
    for i in range(length):
        e=email[i].strip().split(" ")
        l=[x for x in e if x!=" " and x!="" and x in email_bag]     # retaining only the words in the bag
        if l!=[]:
            emails_list.append(l)
            
    X_test=tf_idf_vectorisation(email_bag,emails_list)  #tf-idf vectorisation
    
    return X_test

def main():
    
    lsvc=joblib.load("svm_classifier.sav") #loading the trained linear svm classifier(trained & saved via "Training.ipynb")
    
    X_test=processing().transpose()     #processing+ tf-idf vectorising of test emails
    y_pred=lsvc.predict(X_test).astype("int32")
    print(y_pred)                     #printing the predicted labels
    
    return(y_pred)                    

In [ ]:
y=main()                        #invokes the main() func and label is returned as an array